In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments, pipeline
import torch
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model,TaskType, PromptTuningConfig, PromptTuningInit, PromptEncoderConfig,PromptEncoderReparameterizationType, IA3Config, PrefixTuningConfig
from datasets import Dataset, load_dataset, load_from_disk
# 注意Datacollactor不是在Datasets库里的

d:\Anaconda\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_from_disk("./data/peft/") # 如果用过dataset.save_to_disk("./imdb_dataset")，那再次加载就是用这个方法，disk：磁盘

In [3]:
dataset[0:10]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。',
  '法国的首都是巴黎。',
  '植物: 橡树\n动物:大象\n矿物质:铜矿石',
  '1. 抱歉\n2. 致歉\n3. 认错.',
  'DNA是脱氧核糖核酸(Deoxyribonucleic Acid)的缩写。它是一种生物大分子，负责编码遗传信息并传递遗产特征。 DNA分子由四种核苷酸：腺嘌呤（A），鸟嘌呤（G），胞嘧啶（C）和胸腺嘧啶（T）组成，以特定的序列排列，共同构成某个物种特有的基因组。 DNA的双螺旋结构和碱基配对机制使其能很好地复制和完成遗传信息的转录和翻译。',
  '["hippopotamus", "giraffe", "rhinoceros", "cheetah", "platypus", "parrot", "jellyfish", "octopus", "dolphin", "panther"]',
  '很抱歉，但我无法直接转录录音。您

In [4]:
model = AutoModelForCausalLM.from_pretrained("LiquidAI/LFM2-350M", trust_remote_code=True, device_map="auto", load_in_8bit=True, low_cpu_mem_usage=True,torch_dtype=torch.bfloat16, cache_dir = "./huggingface_cache/LiquidAI/LFM2-350M")
# device_map = "auto" 是自动分配给强大的显卡，和序号无关，比如我的是核显（GPU 0）+独显（GPU 1），就是自动分配到GPU 1上，而不是0上，
# # 多卡情况，可以去掉device_map="auto"，否则会将模型拆开
# llow_cpu_mem_usage=True 用牛逼的技术减少CPU内存的占用，比如分块加载代替一次性加载，比如内存映射
# 启动8bit量化，启动量化之后不能直接微调，要搭载适配器了
# 用bf16精度加载

'''
model = AutoModelForCausalLM.from_pretrained("D:/Pretrained_models/modelscope/Llama-2-13b-ms", low_cpu_mem_usage=True, 
                                             torch_dtype=torch.bfloat16, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16,
                                             bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)
这个是4bit量化的
load_in_4bit=True：以4bit加载，有了这个才有下面的参数
bnb_4bit_compute_dtype=torch.bfloat16：4bit是用来存储时候用的，计算如果用4bit会造成严重的损失，这里设置计算时候用bfloat16
bnb_4bit_quant_type="nf4"：4bit的量化类型，这里选择nf4，基于分位数量化，更均衡
bnb_4bit_use_double_quant=True：4bit的量化时候是否使用双量化，这里选择使用，双量化就进一步压缩，降低显存
'''

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


'\nmodel = AutoModelForCausalLM.from_pretrained("D:/Pretrained_models/modelscope/Llama-2-13b-ms", low_cpu_mem_usage=True, \n                                             torch_dtype=torch.bfloat16, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16,\n                                             bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)\n这个是4bit量化的\nload_in_4bit=True：以4bit加载，有了这个才有下面的参数\nbnb_4bit_compute_dtype=torch.bfloat16：4bit是用来存储时候用的，计算如果用4bit会造成严重的损失，这里设置计算时候用bfloat16\nbnb_4bit_quant_type="nf4"：4bit的量化类型，这里选择nf4，基于分位数量化，更均衡\nbnb_4bit_use_double_quant=True：4bit的量化时候是否使用双量化，这里选择使用，双量化就进一步压缩，降低显存\n'

In [5]:
tokenizer = AutoTokenizer.from_pretrained("LiquidAI/LFM2-350M", trust_remote_code=True, cache_dir = "./huggingface_cache/LiquidAI_tokenizer/LFM2-350M")

# 生成答案检测一下看看是不是加载成功了，这个是github上面的提问模板，复制到这里而已
prompt = "给出保持健康的提示"
input_ids = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    add_generation_prompt=True, # 在用户内容后追加「模型回复前缀」（比如 ### Response 或特殊结束符），让模型知道需要生成回复。
    return_tensors="pt",
    tokenize=True, # 直接完成分词，返回 input_ids，如果不是的话，输入给模型的时候还要解包(前面放**)
    ).to(model.device)

output = model.generate(
    input_ids,
    do_sample=True, # 启用采样策略生成文本（而非贪心搜索），就是说根据概率来采样
    temperature=0.3, # 采样温度，越大越随机
    min_p=0.15, # 采样最小概率，低于此概率的概率会被忽略
    repetition_penalty=1.05, # 重复惩罚，用于防止模型生成重复内容
    max_new_tokens=512,)

print(tokenizer.decode(output[0], skip_special_tokens=False)) # 对output还要解码才能看到汉字

d:\Anaconda\envs\transformers\lib\site-packages\bitsandbytes\autograd\_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


<|startoftext|><|im_start|>user
给出保持健康的提示<|im_end|>
<|im_start|>assistant
保持健康是一个持续的过程，下面是一些建议：

1. **均衡饮食**：确保每日摄入足够的水分、富含蔬菜、水果、全谷物和蛋白质等营养素。避免过量摄入加工食品和糖分。

2. **规律运动**：每周至少进行150分钟的中等强度有氧运动，如快步跑、游泳或骑自行车，每天进行两次。这有助于提高心肺功能、增强肌肉力量和柔韧性。

3. **充足睡眠**：每晚保证7-8小时的高质量睡眠。良好的睡眠习惯有助于身体恢复、情绪调节和整体健康。

4. **保持水分**：每天喝足够的水，尤其是在运动后或在炎热天气外。适量饮用水可以帮助维持体内液体平衡。

5. **戒烟限酒**：长期吸烟和过量饮酒会对健康造成严重损害。寻找健康的替代活动来缓解压力。

6. **定期体检**：定期检查血压、胆固醇、血糖等关键指标，及时发现并处理潜在健康问题。

7. **心理健康**：保持积极的心态，减少压力和焦虑。参与社交活动、冥想或瑜伽等方式可以提升心理健康。

8. **避免过度劳累**：合理安排工作和休息时间，避免过度疲劳和燃尽。适当休息也能促进身体和精神的恢复。

9. **适量摄入抗氧化剂**：富含抗氧化剂的食物，如水果、蔬菜和坚果，可以帮助保护细胞免受自由基损伤。

10. **学习新技能**：不断学习新的知识和技能，可以增加智力活力，提升自信心。

11. **保持社交联系**：与朋友和家人保持良好关系，建立支持网络，这有助于减轻孤独感和压力。

12. **适量使用科技**：尽量减少屏幕时间，保持正确的作息时间，避免长时间处于


In [6]:
# 看一下有多少个参数
all_params = sum(param.numel() for param in model.parameters())
all_params

354483968

In [49]:
# bitfit就是，只对bias进行微调，我这里选择的模型没有bias，所以没办法用bitfit
num_params = 0 # 统计要训练的参数个数
'''
for name, param in model.named_parameters():
    if "bias" not in name:
        param.requires_grad = False
    else:
        param.requires_grad = True
        print(name)
        num_params += param.numel()
for name, param in model.named_parameters():
    print(name, param.requires_grad)
'''

'\nfor name, param in model.named_parameters():\n    if "bias" not in name:\n        param.requires_grad = False\n    else:\n        param.requires_grad = True\n        print(name)\n        num_params += param.numel()\nfor name, param in model.named_parameters():\n    print(name, param.requires_grad)\n'

In [50]:
# 看一下要训练的参数占总的多少
print(num_params/sum(param.numel() for param in model.parameters()))

0.0


In [51]:
print(tokenizer.bos_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id) #第一个也有可能是cls token id，不过我写了输出是None

1 None 0


In [52]:
print(tokenizer(["你好", "你在干什么"],add_special_tokens=True))
print(tokenizer.decode(0))
print(dataset.column_names)
# 打印出来发现这个分词器好像在最后面不会加eos token，这是它自己的问题，和我们无关，处理的时候可以手动加一下

{'input_ids': [[1, 11754, 6400], [1, 11754, 2646, 18759, 22188]], 'attention_mask': [[1, 1, 1], [1, 1, 1, 1, 1]]}
<|pad|>
['output', 'input', 'instruction']


In [53]:
def process_function(example): # example是数据集中的一条数据
    MAX_LENGTH = 256 # 最大长度，要做截断的，这个最大长度越大，显存占用越高
    input_ids = []; labels = []; attention_mask = [] # 不要写成input_ids = [], labels = [], attention_mask = [] 中间写分号不是逗号
    # join方法是字符串的方法，要用字符串.join来调用，然后呢，括号里面是可迭代的对象，最终的功能就是把迭代的对象用字符串连接起来
    question = "\n".join(["Human:"+example["instruction"], example["input"]]) + "\n\nAssistant:" #后续如果数据有空的，要判断一下input是不是空的，不然的话有多余的换行
    question_tokenized = tokenizer(question, add_special_tokens=False) #先不截断，还要合并的
    answer_tokenized = tokenizer(example["output"], add_special_tokens=False) 
    input_ids = [tokenizer.bos_token_id] + question_tokenized["input_ids"] + answer_tokenized["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = [1]*len([tokenizer.bos_token_id]) + question_tokenized["attention_mask"] + answer_tokenized["attention_mask"] + [1]*len([tokenizer.eos_token_id]) #特殊符号给予注意力, 否则模型理解不到文本的结构, 我这个模型没有seq
    labels = [-100]*len([tokenizer.bos_token_id]) + [-100] * len(question_tokenized["input_ids"]) + answer_tokenized["input_ids"] + [-100]*len([tokenizer.eos_token_id]) #让模型专注于预测answer，其他都不用计算损失
    if len(input_ids) > MAX_LENGTH: # 做截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [54]:
processed_dataset = dataset.map(process_function, remove_columns=dataset.column_names) # 把原来数据集的列去掉

In [55]:
tokenizer.decode(processed_dataset[0]["input_ids"]) # 服了总是忘记给input加上双引号

'<|startoftext|>Human:保持健康的三个提示。\n\n\nAssistant:以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。<|im_end|>'

In [56]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)
# 前面我把模型中的所有参数的 requires_grad 设置为 False，因为要搭载适配器，原来模型的都不参与训练

model.embed_tokens.weight True
model.layers.0.conv.conv.weight True
model.layers.0.conv.in_proj.weight False
model.layers.0.conv.out_proj.weight False
model.layers.0.feed_forward.w1.weight False
model.layers.0.feed_forward.w3.weight False
model.layers.0.feed_forward.w2.weight False
model.layers.0.operator_norm.weight True
model.layers.0.ffn_norm.weight True
model.layers.1.conv.conv.weight True
model.layers.1.conv.in_proj.weight False
model.layers.1.conv.out_proj.weight False
model.layers.1.feed_forward.w1.weight False
model.layers.1.feed_forward.w3.weight False
model.layers.1.feed_forward.w2.weight False
model.layers.1.operator_norm.weight True
model.layers.1.ffn_norm.weight True
model.layers.2.self_attn.q_proj.weight False
model.layers.2.self_attn.k_proj.weight False
model.layers.2.self_attn.v_proj.weight False
model.layers.2.self_attn.out_proj.weight False
model.layers.2.self_attn.q_layernorm.weight True
model.layers.2.self_attn.k_layernorm.weight True
model.layers.2.feed_forward.w1.

In [57]:
# lora微调大名鼎鼎，我就不介绍了，网上一大堆
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16, # 这个是lora alpha不是alpha
    lora_dropout=0.05, # 同理，这个是lora dropout不是dropout
    target_modules=["q_proj", "v_proj"], # 看自己模型的层名，也可以用正则匹配
)

config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'v_proj', 'q_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)

In [ ]:
'''
prompt tuning
分为soft prompt和hard prompt

hard prompt：直接对输入的文本提示进行优化调整， 是在自然语言层面去寻找最优的提示词组合。
比如，在进行情感分类任务时，通过人工设计或者启发式搜索算法 ，找到像 “请判断以下文本的情感是积极的还是消极的：[待分类文本]” 这样效果最好的提示表述。

soft prompt：引入一些可训练的连续向量（软提示向量），与输入文本的嵌入表示一起输入到模型中，就是在ebedding前面加了几个能训练的向量而已

总的来说，hard prompt就是死的，不能训练prompt部分，soft prompt就是活的，可以训练。都是改变了模型的架构的

p-tuning 跟这个很像，就是soft prompt本来是对embedding那边进行训练，现在我把embedding那边变了一下，变成一个embedding层加上一个LSTM或者MLP（多层感知机）层，这样便于收敛

prefix-tuning ：transformer是通过前面的词来计算下一个词，所以前面的词复用率很高，用来预测下一个词之后，还要用来预测下下个词，等等
因此会把历史词也拼接到K矩阵，V矩阵前面以加速，而这个prefix方法就差不多这种架构，跟历史词的那种放法一样，每个tranformer块都有
'''

'''
soft_prompt的配置:
config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, # 任务类型
                            prompt_tuning_init=PromptTuningInit.TEXT, # 初始化方式
                            prompt_tuning_init_text="下面是一段人与机器人的对话。", # 初始化文本
                            num_virtual_tokens=len(tokenizer("下面是一段人与机器人的对话。")["input_ids"]), # 虚拟token的数量
                            tokenizer_name_or_path="Langboat/bloom-1b4-zh")
或
config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10)
打印出来参数占比挺低的
'''


In [ ]:
'''
p tuning: 
config = PromptEncoderConfig(task_type=TaskType.CAUSAL_LM, 
                             num_virtual_tokens=10,
                             encoder_reparameterization_type=PromptEncoderReparameterizationType.MLP, # 也可以是LSTM
                             encoder_dropout=0.1, # 训练的时候开启dropout
                             encoder_num_layers=5, # MLP层数是5层
                             encoder_hidden_size=1024 #隐藏层的size
                             )
config
model = get_peft_model(model, config)
print(model.print_trainable_parameters()) # 发现打印出来参数占比还是蛮高的
'''

PromptEncoderConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.P_TUNING: 'P_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, modules_to_save=None, encoder_reparameterization_type=<PromptEncoderReparameterizationType.MLP: 'MLP'>, encoder_hidden_size=1024, encoder_num_layers=5, encoder_dropout=0.1)

In [ ]:
'''
prefix tuning: 
config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10, prefix_projection=True)
当 prefix_projection=True 时，模型会在前缀 token 上添加一个投影层。这个投影层通常由一个或多个线性变换层
（可能还会搭配激活函数，如 GELU 等）组成。它可以将前缀 token 的初始表示变换到一个更适合当前任务的特征空间中，增强前缀 token 的表达能力。
config
model = get_peft_model(model, config)
print(model.print_trainable_parameters()) # 发现打印出来参数占比挺高的，是原来模型的 2.3880%
'''

In [ ]:
'''
IA3 tuning: Infused Adapter by Inhibiting and Amplifying Inner Activations
抑制或放大内部激活，就是给模型里面的激活值乘上一个向量（对应相乘，就是每个分量乘不一样的标量），一般作用的是V矩阵和前馈神经网络的激活值
IA3的训练参数非常的少，效果也不错，少样本学习很好
和lora相比，lora少样本没那么好效果，参数量也达不到那么少，但是lora很灵活，r和alpha可以灵活选择，并且还有qlora这些改进版本
IA3合并之后和lora合并之后速度相差不大，但合并前，IA3因为自己的特性推理速度快，lora没那么快，在需要频繁转变适配器的时候IA3更好（因为这时候不合并，如果合并的话两者差不多）
config = IA3Config(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["feed_forward.w1", "v_proj"],  
    feedforward_modules=["feed_forward.w1"],
)
config
model = get_peft_model(model, config)
print(model.print_trainable_parameters()) # 发现打印出来参数占比很低，0.0055%
'''

In [58]:
model = get_peft_model(model, config)

In [ ]:
model.print_trainable_parameters() # lora下是0.0485%，蛮低的

trainable params: 172,032 || all params: 354,656,000 || trainable%: 0.0485


In [34]:
model 

PeftModelForCausalLM(
  (base_model): IA3Model(
    (model): Lfm2ForCausalLM(
      (model): Lfm2Model(
        (embed_tokens): Embedding(65536, 1024, padding_idx=0)
        (layers): ModuleList(
          (0-1): 2 x Lfm2DecoderLayer(
            (conv): Lfm2ShortConv(
              (conv): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(2,), groups=1024, bias=False)
              (in_proj): Linear8bitLt(in_features=1024, out_features=3072, bias=False)
              (out_proj): Linear8bitLt(in_features=1024, out_features=1024, bias=False)
            )
            (feed_forward): Lfm2MLP(
              (w1): ia3.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=1024, out_features=4608, bias=False)
                (ia3_l): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 1x1024 (cuda:0)])
              )
              (w3): Linear8bitLt(in_features=1024, out_features=4608, bias=False)
              (w2): Linear8bitLt(in_featu

In [ ]:
# 这里，原本模型加载是bf16加载，但是加了config之后，适配器不是bf16的，加上这个让所有的都是bf16，可以进一步降低显存
# 你模型半精度的话，还用adam的话，要把epsilon设置大一点，不然半精度情况下就变成0了，1e-8 = 0
model.bfloat16()

PeftModelForCausalLM(
  (base_model): PeftModelForCausalLM(
    (base_model): IA3Model(
      (model): Lfm2ForCausalLM(
        (model): Lfm2Model(
          (embed_tokens): Embedding(65536, 1024, padding_idx=0)
          (layers): ModuleList(
            (0-1): 2 x Lfm2DecoderLayer(
              (conv): Lfm2ShortConv(
                (conv): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(2,), groups=1024, bias=False)
                (in_proj): Linear8bitLt(in_features=1024, out_features=3072, bias=False)
                (out_proj): Linear8bitLt(in_features=1024, out_features=1024, bias=False)
              )
              (feed_forward): Lfm2MLP(
                (w1): ia3.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=1024, out_features=4608, bias=False)
                  (ia3_l): ParameterDict(  (default): Parameter containing: [torch.cuda.BFloat16Tensor of size 1x1024 (cuda:0)])
                )
                (w3): Linear8bitLt(in_features=102

In [ ]:
config # 配完之后可以看到base model什么的了

IA3Config(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.IA3: 'IA3'>, auto_mapping=None, base_model_name_or_path='LiquidAI/LFM2-350M', revision=None, inference_mode=False, target_modules={'v_proj', 'feed_forward.w1'}, exclude_modules=None, feedforward_modules={'feed_forward.w1'}, fan_in_fan_out=False, modules_to_save=None, init_ia3_weights=True)

In [120]:
args = TrainingArguments(
    output_dir = "./huggingface_cache/LiquidAI/LFM2-350M/checkpoint",
    gradient_accumulation_steps= 10, # 梯度累加，batch设置小，和梯度累加结合用
    gradient_checkpointing= True,
    per_device_train_batch_size= 1, # 这里的batch size是指每个GPU上的训练batch size, 设置小节约显存
    per_device_eval_batch_size= 1,  #评估的batch size
    num_train_epochs= 1, # 训练的epoch数
    optim="adafactor",
    learning_rate= 2e-4,
    logging_steps= 1000, # 日志记录的步数
  # eval_strategy= "steps", # 评估的策略,如果设置为epoch，那eval steps就没用
    save_strategy= "steps", # 保存的策略，如果设置为epoch，那save steps就没用
    save_steps= 10000, # 保存的步数
  # eval_steps= 500, # 评估的步数
    save_total_limit= 2,
)

In [121]:
trainer = Trainer(args = args,
                  model = model,
                  train_dataset = processed_dataset,
                 # eval_dataset = eval_dataset,
                  tokenizer = tokenizer,
                    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, padding = True) # 前面不用paddiding，到这一步再padding也行的，这是自适应padding，根据batch来调整最大长度，你也可以强制是统一长度

) 

C:\Users\86199\AppData\Local\Temp\ipykernel_16924\2118990053.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(args = args,


In [ ]:
model.enable_input_require_grads() # # 开启梯度检查点时，必须要搭配上这个

In [ ]:
# 这里直接训练是不行的，因为我加载的这个模型是纯量化模型，你加载的时候如果选择了量化加载，就相当于量化处理过了，模型变得硬邦邦，参数不好训练了，只能给适配器来训练

In [ ]:
trainer.train() #老人机电脑要两个小时才训练完，这里我就不训练了

d:\Anaconda\envs\transformers\lib\site-packages\bitsandbytes\autograd\_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
medel.save_pretrained(output_dir) # 这个是是保存适配器而不是模型，用了get peft model之后就变成PeftModel了，PeftModel.save_pretrained 是保存适配器

## 下面的不用运行，这个是加载和合并有适配器的模型

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [ ]:
model = AutoModelForCausalLM.from_pretrained("model_name")
tokenizer = AutoTokenizer.from_pretrained("model_name")

In [ ]:
p_model = PeftModel.from_pretrained(model, model_id="./checkpoint-500/") # 使用 Trainer 训练模型时保存的 checkpoint（检查点）文件
# 用p_model推理就是用微调之后的模型推理了，不过还没有合并，会慢一些

In [ ]:
# 这个是合并模型的代码
merge_model = p_model.merge_and_unload()

# 这个保存模型
merge_model.save_pretrained("save_merge_model_path") # 这个就是保存整个模型了

## 对于多种适配器的切换的加载

In [ ]:
# 比如现在你有loraA和loraB，怎么切换这两个适配器，而且有时候禁用适配器呢？
from peft import PeftModel

In [ ]:
# 首先要加载出来主模型
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model1 = PeftModel.from_pretrained(model, model_id="./loraA/", adapter_name="loraA")
# 这个是把loraA适配器装到model上面

In [ ]:
# 有了主模型之后，后面就不需要像上面那么麻烦了，直接用load_adapter()加载adapter就好了
model1.load_adapter("./loraB/", adapter_name="loraB")

In [ ]:
model1.active_adapter #这个是可以看现在激活的是哪个适配器，这里发现还是loraA，说明加载进来不会变适配器

In [ ]:
model1.set_adapter("loraB") # 设置模型适配器为loraB，这里写的是适配器名字

In [ ]:
model.active_adapter # 这时候会发现是loraB

In [ ]:
with model1.disable_adapter():
    # 自己要做的操作